<a href="https://colab.research.google.com/github/hurtadosanti/TrafficSignClassifier/blob/tensorflow2/TrafficSignClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Sign Recognition Classifier
Deep Learning

[Problem Definition](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/Traffic_Sign_Classifier.ipynb)

https://www.tensorflow.org/tutorials/keras/classification#evaluate_accuracy

## Import packages

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
import pickle
import os
from zipfile import ZipFile
from sklearn.utils import shuffle
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
print(tf.__version__)
%matplotlib inline


2.3.0


## Load Data

In [20]:
traffic_signs_file = 'traffic-signs-data.zip'
!wget --no-check-certificate --progress=bar:force:noscroll \
    https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip \
    -O traffic-signs-data.zip

!wget --no-check-certificate --progress=bar:force:noscroll \
    https://raw.githubusercontent.com/udacity/CarND-Traffic-Sign-Classifier-Project/master/signnames.csv -O signnames.csv


--2020-09-29 19:59:56--  https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.248
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  97.8MB/s    in 1.2s    

2020-09-29 19:59:57 (97.8 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]

--2020-09-29 19:59:57--  https://raw.githubusercontent.com/udacity/CarND-Traffic-Sign-Classifier-Project/master/signnames.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 999 [text

In [21]:
with ZipFile(traffic_signs_file, 'r') as zipObj:
   zipObj.extractall()
   print('Finish extracting')

Finish extracting


In [22]:
train = pickle.load(open("train.p", 'rb'))
valid=pickle.load(open("valid.p", 'rb'))
test = pickle.load(open("test.p", 'rb'))
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']
# Verify that the data is complete
assert(len(X_train)==len(y_train))
assert(len(X_valid)==len(y_valid))
assert(len(X_test)==len(y_test))

assert(len(X_train)>0)
assert(len(X_valid)>0)
assert(len(X_test)>0)
print('Data is correct')

Data is correct


## Data preprocessing

In [23]:
X_train_normalized = X_train/255.0
X_valid_normalized = X_valid/255.0
X_train_shuffle, y_train_shuffle  = shuffle(X_train_normalized, y_train)
image_shape = X_train[0].shape

## Training
### Network Architecture

### Prepare the model

In [25]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(43,activation='sigmoid'))


### Execute the training
We stop when accuracy is over 0.95

In [26]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)             

In [31]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_normalized, y_train, epochs=10,batch_size=128, 
                    validation_data=(X_valid_normalized, y_valid))


Epoch 1/10
272/272 [==============================] - 2s 7ms/step - loss: 2.9137 - accuracy: 0.7981 - val_loss: 2.9378 - val_accuracy: 0.8034
Epoch 2/10
272/272 [==============================] - 2s 6ms/step - loss: 2.9122 - accuracy: 0.8028 - val_loss: 2.9363 - val_accuracy: 0.8107
Epoch 3/10
272/272 [==============================] - 2s 6ms/step - loss: 2.9106 - accuracy: 0.8105 - val_loss: 2.9377 - val_accuracy: 0.7971
Epoch 4/10
272/272 [==============================] - 2s 6ms/step - loss: 2.9097 - accuracy: 0.8139 - val_loss: 2.9371 - val_accuracy: 0.8186
Epoch 5/10
272/272 [==============================] - 2s 6ms/step - loss: 2.9098 - accuracy: 0.8154 - val_loss: 2.9399 - val_accuracy: 0.8168
Epoch 6/10
272/272 [==============================] - 2s 6ms/step - loss: 2.9083 - accuracy: 0.8213 - val_loss: 2.9388 - val_accuracy: 0.8172
Epoch 7/10
272/272 [==============================] - 2s 6ms/step - loss: 2.9081 - accuracy: 0.8196 - val_loss: 2.9379 - val_accuracy: 0.8263
Epoch 

## Test the model

In [32]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

395/395 - 1s - loss: 2.9260 - accuracy: 0.7939

Test accuracy: 0.7939034104347229
